<a href="https://colab.research.google.com/github/Sewlell/VoiceCraft/blob/master/VoiceCraftFinetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/jasonppy/VoiceCraft

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# ---------------------- Currently this Colab doesn't have saving model feature yet. Only remove the below # if you wanted to add yourself.

# from google.colab import drive
# drive.mount('/content/drive')

## Rerun this cell twice. And let it restart.

In [ ]:
!pip install -e git+https://github.com/facebookresearch/audiocraft.git@c5157b5bf14bf83449c17ea1eeb66c19fb4bc7f0#egg=audiocraft
!pip install xformers==0.0.22
!pip install torchaudio==2.0.2 torch==2.0.1 # this assumes your system is compatible with CUDA 11.7, otherwise checkout https://pytorch.org/get-started/previous-versions/#v201
!apt-get install ffmpeg # if you don't already have ffmpeg installed
!apt-get install espeak-ng # backend for the phonemizer installed below
!pip install tensorboard==2.16.2
!pip install phonemizer==3.2.1
!pip install datasets==2.16.0
!pip install torchmetrics==0.11.1
!pip install huggingface_hub==0.22.2

In [ ]:
from google.colab import files

# Upload the audio file to Google Drive
uploaded = files.upload()

audio_path = next(iter(uploaded))

# Print the path of the uploaded audio file
print(f"Uploaded audio file: {audio_path}")


In [46]:
#@title ## Create Metadata

#@markdown Size of Gigaspeech, `xl` is used for VoiceCraft training and `xs` is good for debugging
dataset_size = "xs" # @param ["xs", "s", "m", "l", "xl"]
#@markdown Number of samples in each mega batch for multiprocess dataloading
mega_batch_size = 100 # @param {type:"integer"}
#@markdown Number of samples in each batch for singleprocess dataloading
number_of_worker = 4 # @param {type:"integer"}
#@markdown Batch size for encodec encoding, decrease it if OOM.
batch_size = 6 # @param {type: "slider", min: 1, max: 32}
#@markdown Encodec input audio sample rate
audio_sample_rate = "41000" # @param ["16000", "40000", "41000"] {allow-input: true}
#@markdown [ADVANCED] Encodec downsample rate (Default: 320)
downsample_rate = 320 # @param {type:"integer"}
#@markdown [ADVANCED] Encodec model code sample rate (Default: 50)
modelcode_sr = 50 # @param {type:"integer"}
#@markdown Drop audios that are longer than this number ( in second )
len_cap = 35.0 # @param {type:"number"}
#@markdown Max length of audio in samples, if exceed, will cut a batch into half to process, decrease this number if OOM on your machine ( in second )
max_len = 30000 # @param {type:"integer"}


In [ ]:
#@markdown If you use Gigaspeech dataset or other dataset that required you to sign an agreement in order to download the dataset. Please run this cell.

#@markdown When it ask for your access token, give it the token which you can get it here (https://huggingface.co/settings/tokens). Also it should be read, not write.

import os

# Set the environment variable 'USER' to the current user's name.
os.environ["USER"] = os.getenv("USER", "user")

!huggingface-cli login

In [ ]:
import os
import shutil

!wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/encodec_4cb2048_giga.th -O encodec_4cb2048_giga.th
destination_dir = "/content/VoiceCraft/data/"
os.makedirs(destination_dir, exist_ok=True)
shutil.move("encodec_4cb2048_giga.th", os.path.join(destination_dir, "encodec_4cb2048_giga.th"))

# Ignore `words count mismatch` warning!

### Run this if you wanted to run on Gigaspeech dataset.

In [ ]:
!python /content/VoiceCraft/data/phonemize_encodec_encode_hf.py \
--dataset_size {dataset_size} \
--download_to /content/VoiceCraft/data/scratch/pyp/datasets/gigaspeech_debug \
--save_dir /content/VoiceCraft/data/scratch/pyp/datasets/gigaspeech_phn_enc_manifest_debug \
--encodec_model_path /content/VoiceCraft/data/encodec_4cb2048_giga.th \
--n_workers {number_of_worker} \
--mega_batch_size {mega_batch_size} \
--batch_size {batch_size} \
--model_sr {audio_sample_rate} \
--downsample_rate {downsample_rate} \
--model_code_sr {modelcode_sr} \
--len_cap {len_cap} \
--max_len {max_len}


In [ ]:
import os
import shutil

# Create the destination directory if it doesn't exist
destination_dir = "/content/VoiceCraft/data/scratch/pyp/datasets/gigaspeech_phn_enc_manifest_debug/manifest/"
os.makedirs(destination_dir, exist_ok=True)

# Download the file from Hugging Face
!wget https://huggingface.co/datasets/pyp1/VoiceCraft_RealEdit/resolve/main/train.txt -O train.txt
!wget https://huggingface.co/datasets/pyp1/VoiceCraft_RealEdit/resolve/main/validation.txt -O validation.txt
# This as we finetune VoiceCraft pretrained
!wget https://huggingface.co/datasets/pyp1/VoiceCraft_RealEdit/resolve/main/vocab.txt -O vocab.txt

# Move the downloaded file to the destination directory
shutil.move("train.txt", os.path.join(destination_dir, "train.txt"))
shutil.move("validation.txt", os.path.join(destination_dir, "validation.txt"))
shutil.move("vocab.txt", os.path.join(destination_dir, "vocab.txt"))


In [ ]:
#@title ## Start Training
!bash /content/VoiceCraft/z_scripts/e830M_ft.sh

# ^^ Turn 'e830M.sh' to 'e830_ft.sh' if you wished to use that finetuned version.

In [ ]:
from google.colab import files

# Download the file
files.download('train.txt')

# ------------ ^^^^^^^^^^ Replace this to the model path.